In [1]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [2]:
# import all your modules here
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

In [3]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY

In [4]:
print(baseline)
print(study)

['2022-10-01', '2022-12-31']
['2023-01-01', '2023-01-31']


In [5]:
df_devices = ubi.get_available_devices_v2(label='bancolombia', level='group', page_size=1000)

In [6]:
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [7]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [8]:
CHUNK_SIZE = 1
DATE_INTERVAL_REQUEST = {'start': baseline[0], 'end': study[1]}

df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Baseline:{baseline}, Study:{study}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    

    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )

    current_idx = idx_end+1
    
    if (current_idx > n_vars):
        current_idx = n_vars

    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")

    if response.status_code != 204:

        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")

df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)


Making request for the following interval: Baseline:['2022-10-01', '2022-12-31'], Study:['2023-01-01', '2023-01-31']
Progress: 0.6%
Progress: 1.0%
Progress: 1.3%
Progress: 1.6%
Progress: 1.9%
Progress: 2.2%
Progress: 2.6%
Progress: 2.9%
Progress: 3.2%
Progress: 3.5%
Progress: 3.8%
Progress: 4.2%
Progress: 4.5%
Progress: 4.8%
Progress: 5.1%
Progress: 5.4%
Progress: 5.8%
Progress: 6.1%
Progress: 6.4%
Progress: 6.7%
Progress: 7.0%
Progress: 7.3%
Progress: 7.7%
Progress: 8.0%
Progress: 8.3%
Progress: 8.6%
Progress: 8.9%
Progress: 9.3%
Progress: 9.6%
Progress: 9.9%
Progress: 10.2%
Progress: 10.5%
Progress: 10.9%
Progress: 11.2%
Progress: 11.5%
Progress: 11.8%
Progress: 12.1%
Progress: 12.5%
Progress: 12.8%
Progress: 13.1%
Progress: 13.4%
Progress: 13.7%
Progress: 14.1%
Progress: 14.4%
Progress: 14.7%
Progress: 15.0%
Progress: 15.3%
Progress: 15.7%
Progress: 16.0%
Progress: 16.3%
Progress: 16.6%
Progress: 16.9%
Progress: 17.3%
Progress: 17.6%
Progress: 17.9%
Progress: 18.2%
Progress: 18.5%
P

In [ ]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)